# Purpose:
- Quickly compare between pan-neuronal expression strategies.
- Mainly gathering experiment directory paths

In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from brain_observatory_qc.data_access import from_lims

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc

from pymongo import MongoClient
mongo = MongoClient("flaskapp.corp.alleninstitute.org", 27017)



In [1]:
# helper function
def get_session_info_per_group_of_mice(mouse_ids, gcamp_type):
    experiment_table = pd.DataFrame()
    problem_experiments = []
    for mouse_id in mouse_ids:
        mouse_data = from_lims.get_imaging_ids_for_mouse_id(mouse_id)
        if len(mouse_data) > 0:
            ophys_experiment_ids = mouse_data.ophys_experiment_id.values
            for ophys_experiment_id in ophys_experiment_ids: 
                try: 
                    expt_info = from_lims.get_general_info_for_ophys_experiment_id(ophys_experiment_id)
                    if len(expt_info) > 0:
                        genotype = from_lims.get_genotype_for_ophys_experiment_id(ophys_experiment_id)
                        expt_info.insert(0, "mouse_id", mouse_id ) # expt_info only contains `donor_id` and `specimen_id`, need to add mouse_id here
                        expt_info.insert(1, "full_genotype", genotype) # base function doesnt get genotype, add it
                        experiment_table = pd.concat([experiment_table, expt_info])
                except: 
                    # print('problem for mouse_id: ', mouse_id, ', expt_id: ', ophys_experiment_id)
                    problem_experiments.append(ophys_experiment_id)
    experiment_table['gcamp'] = gcamp_type
    return experiment_table, problem_experiments


def get_mongo_client(username, password, host, port):
    client = MongoClient(f'mongodb://{username}:{password}@{host}:{port}')
    return client


def get_zdrift(opid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    opid = int(opid)
    try:
        record = client.records.metrics.find_one({'data_id': opid})
    except:
        raise('No record found for opid: ', opid)
        
    try:
        zdrift = record['local_z_stack']['z_drift_corr_um_diff']
    except:
        zdrift = np.nan
    return zdrift


def get_monitor_sync(osid):
    try:
        client = get_mongo_client('public', 'public_password', 'qc-sys-db', 27017)
    except:
        raise('Failed to connect to mongo')
    
    osid = int(osid)
    try:
        record = client.records.metrics.find_one({'data_id': osid})
    except:
        raise('No record found for osid: ', osid)
        
    try:
        monitor_sync = record['rig_sync']['display_lag']
    except:
        monitor_sync = np.nan
    return monitor_sync

In [3]:
mids_ribo_aav_local = [719364, 726087, 719363]
mids_ribo_aav_ro = [730929, 730932, 730933]
mids_ribo_aav_icv = [738331, 738332, 738333]
mids_snap25_oi4_dox = [726433]
mids_oi4_homo = []

In [22]:
gcamp_table = pd.DataFrame()
problems_table = []

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_local, 'ribo_aav_local')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_ro, 'ribo_aav_ro')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_ribo_aav_icv, 'ribo_aav_icv')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_snap25_oi4_dox, 'snap25_oi4')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

temp_table, temp_problems = get_session_info_per_group_of_mice(mids_oi4_homo, 'oi4_homo')
gcamp_table = pd.concat([gcamp_table, temp_table])
problems_table.extend(temp_problems)

print(len(problems_table))

In [23]:
print(len(problems_table))

0


In [24]:
gcamp_table['mouse_id'] = gcamp_table['mouse_id'].astype(int)
gcamp_table['ophys_experiment_id'] = gcamp_table['ophys_experiment_id'].astype(int)
gcamp_table['ophys_session_id'] = gcamp_table['ophys_session_id'].astype(int)
gcamp_table['depth'] = gcamp_table['depth'].astype(int)
depth_divider = [0, 125, 225, 325, 500]
depth_labels = [75, 175, 275, 375]
gcamp_table['target_depth'] = pd.cut(gcamp_table.depth, bins=depth_divider, labels=depth_labels, right=False)
gcamp_table.head(3)

,mouse_id,full_genotype,ophys_experiment_id,ophys_session_id,behavior_session_id,foraging_id,ophys_container_id,supercontainer_id,experiment_workflow_state,session_workflow_state,...,equipment_name,project,experiment_storage_directory,behavior_storage_directory,session_storage_directory,container_storage_directory,supercontainer_storage_directory,specimen_storage_directory,gcamp,target_depth
0,719364,wt/wt,1363365762,1363258688,1.363273e+09,4ad6ce93-d4d7-4f87-a0df-63c21756ffb3,1.374101e+09,None,failed,uploaded,...,MESO.2,LearningmFISHDevelopment,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...,ribo_aav_local,275
0,719364,wt/wt,1363365760,1363258688,1.363273e+09,4ad6ce93-d4d7-4f87-a0df-63c21756ffb3,1.374101e+09,None,passed,uploaded,...,MESO.2,LearningmFISHDevelopment,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...,ribo_aav_local,175
0,719364,wt/wt,1346373273,1346243082,1.346263e+09,1fdc1889-a2c9-465e-b2a8-2d6a0b5591d4,1.374101e+09,None,failed,uploaded,...,MESO.2,LearningmFISHDevelopment,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,\\allen\programs\mindscope\production\learning...,None,\\allen\programs\mindscope\production\learning...,ribo_aav_local,375


In [26]:
target_depth = 375
temp_table = gcamp_table.query('mouse_id==726433 and target_depth==@target_depth').sort_values('date_of_acquisition')
# temp_table
print(temp_table.iloc[-1].experiment_storage_directory / 'processed')

\\allen\programs\mindscope\production\learning\prod1\specimen_1341432280\ophys_session_1372254967\ophys_experiment_1372469071\processed
